In [ ]:
from typing import Dict
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
import lightgbm as lgb

from config import CONFIG
from data_manager import DataManager
from metric import eval_metric, xgb_eval_metric, lgb_eval_metric

In [ ]:
manager = DataManager()

In [ ]:
train_data, valid_data = manager.load_weather_data(mode="train", split=True)
train_data = manager.apply_feature_engineering(train_data)
valid_data = manager.apply_feature_engineering(valid_data)

In [ ]:
test_data = manager.load_weather_data(mode="test")
test_data = manager.apply_feature_engineering(test_data)

In [ ]:
train_target_data, valid_target_data = manager.load_target_data(mode="train", split=True)
test_target_data = manager.load_target_data(mode="test")

In [ ]:
train_data: Dict[int, pl.DataFrame] = manager.get_ml_data(train_data, sourece="NWP_21")
valid_data: Dict[int, pl.DataFrame] = manager.get_ml_data(valid_data, sourece="NWP_21")
test_data: Dict[int, pl.DataFrame] = manager.get_ml_data(test_data, sourece="NWP_21")

In [ ]:
def cal_time_feature():
    
    return [pl.col("forecast_time_right").dt.quarter().alias("quarter"),
            pl.col("forecast_time_right").dt.month().alias("month"),
            pl.col("forecast_time_right").dt.day().alias("day"),
            pl.col("forecast_time_right").dt.hour().alias("hour"),   
            pl.col("forecast_time_right").dt.minute().alias("minute")]

train_data_list = []
valid_data_list = []
test_data_list = []
for plant_id in range(1, 11):
    
    train_df = train_data[plant_id]
    train_target = train_target_data[plant_id]

    valid_df = valid_data[plant_id]
    valid_target = valid_target_data[plant_id]
    
    test_df = test_data[plant_id]
    test_target_ = test_target_data[plant_id]
    
    train = (
        train_df.join(train_target, on="forecast_hour", how="right")
        .with_columns(
            cal_time_feature()
        )
    )
    valid = (
        valid_df
        .join(valid_target, on="forecast_hour", how="right")
        .with_columns(
            cal_time_feature()
        )
    )
    test = (
        test_df
        .join(test_target_, on="forecast_hour", how="right")
        .with_columns(
            cal_time_feature()
        )
    )
    train_data_list.append(train)
    valid_data_list.append(valid)
    test_data_list.append(test)
input_train_data = (
    pl.concat(train_data_list)
    .sort(["plant_id", "forecast_time"])
)
input_valid_data = (
    pl.concat(valid_data_list)
    .sort(["plant_id", "forecast_time"])
)
input_test_data = (
    pl.concat(test_data_list)
    .sort(["plant_id", "forecast_time"])
)
input_test_data

In [ ]:
input_test_data

In [ ]:
input_train_data.shape

In [ ]:
input_train_data.shape, input_valid_data.shape

In [ ]:
input_train_data.null_count()

In [ ]:
space_features = [
    f"{feature}_{stat}" for feature 
    in CONFIG.FEATURES for stat in ["mean", "max", "min", "std"]
]
# time_features = [f"{f}_mean_4" for f in CONFIG.FEATURES] + [f"{f}_max_4" for f in CONFIG.FEATURES] + [f"{f}_min_4" for f in CONFIG.FEATURES] + [f"{f}_std_4" for f in CONFIG.FEATURES]
new_feature = space_features + ["quarter", "month", "day", "hour", "minute", "plant_id"]
GBDT_FASTURES = CONFIG.FEATURES + new_feature

In [ ]:
len(GBDT_FASTURES)

In [ ]:
xgb_dtrain = xgb.DMatrix(
    data = input_train_data.select(GBDT_FASTURES).to_numpy(),
    label = input_train_data["target"].to_numpy()
)

xbg_dvalid = xgb.DMatrix(
    data = input_valid_data.select(GBDT_FASTURES).to_numpy(),
    label = input_valid_data["target"].to_numpy()
)

xgb_dtest = xgb.DMatrix(
    data = input_test_data.select(GBDT_FASTURES).to_numpy(),
    label = input_test_data["target"].to_numpy()
)

In [ ]:
xgb_model = xgb.train(
    params=CONFIG.XGB_PARAMETER, 
    dtrain=xgb_dtrain, 
    num_boost_round=350, 
    evals=[(xgb_dtrain, "train"), (xbg_dvalid, "valid")], 
    custom_metric=xgb_eval_metric)

In [ ]:
feature_mapping = {}

for i, feature in enumerate(GBDT_FASTURES):
    feature_mapping[f"f{i}"] = feature
print(feature_mapping)

In [ ]:
xgb_feature_importance =  xgb_model.get_score(importance_type='total_gain') 
feature_gain_maping = {}
for key, feature in feature_mapping.items():
    if key in xgb_feature_importance:
        feature_gain_maping[feature] = xgb_feature_importance[key]
sorted_feature_gain = sorted(feature_gain_maping.items(), key=lambda x: x[1], reverse=True)

In [ ]:
# 拆分特征与增益
features = [x[0] for x in sorted_feature_gain]
gains = [x[1] for x in sorted_feature_gain]

sns.set_theme(style="whitegrid")
plt.figure(figsize=(10, max(6, len(features[:30]) * 0.4)), dpi=400)
sns.barplot(x=gains[:30], y=features[:30], hue=features[:30], palette="viridis", legend=False)
plt.xlabel("Total Gain")
plt.ylabel("Feature")
plt.title("XGBoost Top 30 Feature Importance by Total Gain", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
lgb_dtrain = lgb.Dataset(
    input_train_data.select(GBDT_FASTURES).to_numpy(),
    label=input_train_data["target"].to_numpy()
)
lgb_dvalid = lgb.Dataset(
    input_valid_data.select(GBDT_FASTURES).to_numpy(),
    label=input_valid_data["target"].to_numpy()
)
lgb_dtest = lgb.Dataset(
    input_test_data.select(GBDT_FASTURES).to_numpy(),
    label = input_test_data["target"].to_numpy()
)

In [ ]:
lgb_model = lgb.train(
    CONFIG.LGB_PARAMETER,
    lgb_dtrain,
    num_boost_round =350,
    valid_sets=[lgb_dtrain, lgb_dvalid],
    feval=lgb_eval_metric,
    callbacks=[lgb.early_stopping(350), lgb.log_evaluation(10)],
)

### Ensemble Valid Score

In [ ]:
input_valid_data = (
    input_valid_data
    .with_columns([
        pl.Series("xgb_pred", xgb_model.predict(xbg_dvalid)),
        pl.Series("lgb_pred", 
                  lgb_model.predict(
                      input_valid_data
                      .select(GBDT_FASTURES)
                      .to_numpy())),
    ])
    .with_columns([
        ((pl.col("xgb_pred") + pl.col("lgb_pred")) / 2).alias("ensemble_pred")
    ])
)

xgb_score = eval_metric(input_valid_data["xgb_pred"].to_numpy(), input_valid_data["target"].to_numpy())
lgb_score = eval_metric(input_valid_data["lgb_pred"].to_numpy(), input_valid_data["target"].to_numpy())
score = eval_metric(input_valid_data["ensemble_pred"].to_numpy(), input_valid_data["target"].to_numpy())
print(f"xgb_score: {xgb_score}, lgb_score: {lgb_score}, ensemble_score: {score}")

In [ ]:
# 不标准化
# xgb_score: 0.6741573389807871(0.6401), lgb_score: 0.6717744979993772, ensemble_score: 0.674451319997887(0.6388)
# 标准化
# xgb_score: 0.6689230852721415(0.6324), lgb_score: 0.6680989689556682, ensemble_score: 0.669423321209362, 
# 不标准化+空间特征(mean_std)
# xgb_score: 0.6835227219213333(0.6420), lgb_score: 0.6771717881299739, ensemble_score: 0.6819068042032914
# 不标准化+空间特征(mean_std_max_min)
# xgb_score: 0.6827632113216081(0.6424), lgb_score: 0.6760188949448137, ensemble_score: 0.6811945546478921(0.6417)


In [ ]:
predict = (
    xgb_model.predict(xgb_dtest) 
     + lgb_model.predict(input_test_data.select(GBDT_FASTURES).to_numpy())
) / 2
pred_df = (
    input_test_data
    .with_columns([
        pl.Series("predict", predict.flatten()),
    ])
    .rename({
        "forecast_time_right": "Time",
        "predict": "Power",
    })
    .select("Time", "Power", "plant_id")
)

# 保存提交文件
for (plant_id, ), df in pred_df.group_by("plant_id"):
    df = (
        df.with_columns([
            pl.col("Time").dt.strftime("%Y/%-m/%-d %H:%M").alias("Time")
        ])
        .drop("plant_id")
        .to_pandas()
        .set_index("Time")
    )
    df.to_csv(f"{CONFIG.OUTPUT_PATH}/output{plant_id}.csv")